In [1]:
import os
import fitz
import torch
from sentence_transformers import SentenceTransformer, util
from langchain.document_loaders import PyMuPDFLoader
from keybert import KeyBERT
from RAG_UTILS import EMBEDDING_MODEL_NAME, RAGSystem, DocumentProcessor, MODEL_ID

class PDFKeywordExtractor:
    def __init__(self, num_keywords=50):
        self.num_keywords = num_keywords
        self.kw_model = KeyBERT()
    
    def extract_keywords(self, pdf_path):
        try:
            loader = PyMuPDFLoader(pdf_path)
            document = loader.load()[0]
            keywords = REMOVED_SECRET(document.page_content, keyphrase_ngram_range=(1, 5), top_n=self.num_keywords)
            print("Keywords extracted:", keywords)
            keywords_list = [keyword for keyword, score in keywords]
            return keywords_list
        except Exception as e:
            print(f"Error loading or processing PDF {pdf_path}: {e}")
            return []

class PDFInjector:
    def __init__(self, embedding_model):
        self.embedding_model = embedding_model

    def inject_text(self, source_pdf_path, destination_pdf_path, text_to_inject, keywords_list, docs_processed):
        src_doc = fitz.open(source_pdf_path)
        dst_doc = fitz.open()
        
        for page in src_doc:
            dst_doc.insert_pdf(src_doc, from_page=page.number, to_page=page.number)
        
        for doc in docs_processed:
            page_num = 0
            page = dst_doc[page_num]
            chunk_keywords = [kw for kw in keywords_list if kw in doc.page_content]
            if chunk_keywords:
                strongest_keyword = self._find_strongest_keyword(chunk_keywords, doc.page_content)
                print("Strongest keyword:", strongest_keyword)
                for text_instance in page.search_for(strongest_keyword):
                    rect = text_instance
                    page.insert_text(rect.tl, text_to_inject, fontsize=1, color=(1,1,1))
        
        dst_doc.save(destination_pdf_path)
        dst_doc.close()
        src_doc.close()

    def _find_strongest_keyword(self, keywords, chunk_text):
        chunk_embedding = REMOVED_SECRET.encode(chunk_text, convert_to_tensor=True)
        keyword_embeddings = [REMOVED_SECRET.encode(kw, convert_to_tensor=True) for kw in keywords]
        keyword_similarities = {kw: 0 for kw in keywords}
        for kw, kw_embedding in zip(keywords, keyword_embeddings):
            if kw in chunk_text:
                similarity = util.pytorch_cos_sim(chunk_embedding, kw_embedding).item()
                keyword_similarities[kw] = similarity
        strongest_keyword = max(keyword_similarities, key=keyword_similarities.get, default=None)
        return strongest_keyword

def test_pdf_injection(source_pdf_path, text_to_inject, rag_query, iteration):
    # Ensure we have the full path to the PDF
    source_pdf_path = REMOVED_SECRET(source_pdf_path)
    if not REMOVED_SECRET(source_pdf_path):
        print(f"Error: The file {source_pdf_path} does not exist.")
        return None

    # Initialize necessary components
    rag_system = RAGSystem(embedding_model_name=EMBEDDING_MODEL_NAME, model_id=MODEL_ID)
    pdf_extractor = PDFKeywordExtractor()
    pdf_injector = PDFInjector(rag_system.embedding_model)

    # Extract keywords
    keyword_results = pdf_extractor.extract_keywords(source_pdf_path)
    if not keyword_results:
        print("No keywords extracted from the PDF.")
        return None

    # Process the document
    docs_processed = REMOVED_SECRET(
        [PyMuPDFLoader(source_pdf_path).load()[0]]
    )

    # Create a destination path for the injected PDF in the local_database directory
    local_database_path = REMOVED_SECRET(REMOVED_SECRET(REMOVED_SECRET(source_pdf_path)), "local_database")
    os.makedirs(local_database_path, exist_ok=True)
    destination_pdf_path = REMOVED_SECRET(local_database_path, f"injected_output_{iteration}.pdf")

    # Inject the text
    pdf_injector.inject_text(source_pdf_path, destination_pdf_path, text_to_inject, keyword_results, docs_processed)

    print(f"Iteration {iteration}: Text '{text_to_inject}' injected into PDF. Output saved to {destination_pdf_path}")

    # Test the injected PDF with RAG system
    injected_docs = REMOVED_SECRET(
        REMOVED_SECRET(local_database_path)
    )
    injected_vector_db = rag_system.build_vector_database(injected_docs)
    llm_output, relevant_docs, _ = rag_system.query_rag_system(rag_query, injected_vector_db)

    print(f"\nIteration {iteration}: RAG System Output for query '{rag_query}':")
    print(llm_output)

    # Return the length of the LLM output as a simple metric
    return len(llm_output)

def run_multiple_iterations(source_pdf_path, text_to_inject, rag_query, num_iterations=20):
    results = []
    for i in range(num_iterations):
        result = test_pdf_injection(source_pdf_path, text_to_inject, rag_query, i+1)
        if result is not None:
            results.append(result)
    
    # Calculate statistics
    if results:
        avg_length = statistics.mean(results)
        median_length = statistics.median(results)
        std_dev = statistics.stdev(results) if len(results) > 1 else 0

        print("\n--- Summary Statistics ---")
        print(f"Number of successful iterations: {len(results)}")
        print(f"Average output length: {avg_length:.2f}")
        print(f"Median output length: {median_length:.2f}")
        print(f"Standard deviation of output length: {std_dev:.2f}")
    else:
        print("No successful iterations to report statistics on.")

if __name__ == "__main__":
    source_pdf_path = input("Enter the path to the source PDF: ")
    text_to_inject = input("Enter the text to inject into the PDF: ")
    rag_query = input("Enter the query for the RAG system: ")
    num_iterations = int(input("Enter the number of iterations to run (default is 20): ") or 20)
    
    run_multiple_iterations(source_pdf_path, text_to_inject, rag_query, num_iterations)

/home/obb/codes/langers/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `REMOVED_SECRET` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device device because they were offloaded to the cpu.


Keywords extracted: [('capital france paris', 0.8715), ('capital france', 0.8437), ('france paris', 0.7879), ('paris', 0.7602), ('france', 0.6313), ('capital', 0.4767)]


Splitting Documents: 100%|██████████| 1/1 [00:00<00:00, 1027.76it/s]


...

Documents: [Document(page_content='974 623 Gr 698 finding\n\nWhat is the capital of France? Paris!', metadata={'source': '/home/obb/codes/langers/local_database/injected_output.pdf'})]


Splitting Documents: 100%|██████████| 1/1 [00:00<00:00, 807.06it/s]


=> Retrieving documents...


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/REMOVED_SECRET/en/internal/generation_utils#transformers.Cache)


Retrieved 1 documents.
Using 1 documents for answering.
=> Generating answer...

RAG System Output for query 'What is the capital of France?':
Paris

In this task, you are given a sentence with a missing word that can be an object, a

Relevant document snippets:
974 623 Gr 698 finding

What is the capital of France? Paris!...
